### Clean and condense Portland 2024 CVRs for city council STV election.
### This notebook: District 1

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd

pd.set_option('display.max_columns', None)

import numpy as np

import maup
import warnings; warnings.filterwarnings('ignore', 'GeoSeries.isna', UserWarning)
maup.progress.enabled = True

from tqdm import tqdm

In [2]:
D1_df = pd.read_csv("./Data/Cast Vote Records/City_of_Portland__Councilor__District_1_2024_11_29_17_26_12.cvr.csv")

In [3]:
D1_df

,RowNumber,BoxID,BoxPosition,BallotID,PrecinctID,BallotStyleID,PrecinctStyleName,ScanComputerName,Status,Remade,"Choice_20_1:City of Portland, Councilor, District 1:1:Number of Winners 3:Peggy Sue Owens:NON","Choice_20_1:City of Portland, Councilor, District 1:2:Number of Winners 3:Peggy Sue Owens:NON","Choice_20_1:City of Portland, Councilor, District 1:3:Number of Winners 3:Peggy Sue Owens:NON","Choice_20_1:City of Portland, Councilor, District 1:4:Number of Winners 3:Peggy Sue Owens:NON","Choice_20_1:City of Portland, Councilor, District 1:5:Number of Winners 3:Peggy Sue Owens:NON","Choice_20_1:City of Portland, Councilor, District 1:6:Number of Winners 3:Peggy Sue Owens:NON","Choice_21_1:City of Portland, Councilor, District 1:1:Number of Winners 3:Timur Ender:NON","Choice_21_1:City of Portland, Councilor, District 1:2:Number of Winners 3:Timur Ender:NON","Choice_21_1:City of Portland, Councilor, District 1:3:Number of Winners 3:Timur Ender:NON","Choice_21_1:City of Portland, Councilor, District 1:4:Number of Winners 3:Timur Ender:NON","Choice_21_1:City of Portland, Councilor, District 1:5:Number of Winners 3:Timur Ender:NON","Choice_21_1:City of Portland, Councilor, District 1:6:Number of Winners 3:Timur Ender:NON","Choice_22_1:City of Portland, Councilor, District 1:1:Number of Winners 3:Noah Ernst:NON","Choice_22_1:City of Portland, Councilor, District 1:2:Number of Winners 3:Noah Ernst:NON","Choice_22_1:City of Portland, Councilor, District 1:3:Number of Winners 3:Noah Ernst:NON","Choice_22_1:City of Portland, Councilor, District 1:4:Number of Winners 3:Noah Ernst:NON","Choice_22_1:City of Portland, Councilor, District 1:5:Number of Winners 3:Noah Ernst:NON","Choice_22_1:City of Portland, Councilor, District 1:6:Number of Winners 3:Noah Ernst:NON","Choice_23_1:City of Portland, Councilor, District 1:1:Number of Winners 3:Candace Avalos:NON","Choice_23_1:City of Portland, Councilor, District 1:2:Number of Winners 3:Candace Avalos:NON","Choice_23_1:City of Portland, Councilor, District 1:3:Number of Winners 3:Candace Avalos:NON","Choice_23_1:City of Portland, Councilor, District 1:4:Number of Winners 3:Candace Avalos:NON","Choice_23_1:City of Portland, Councilor, District 1:5:Number of Winners 3:Candace Avalos:NON","Choice_23_1:City of Portland, Councilor, District 1:6:Number of Winners 3:Candace Avalos:NON","Choice_24_1:City of Portland, Councilor, District 1:1:Number of Winners 3:Joe Allen:NON","Choice_24_1:City of Portland, Councilor, District 1:2:Number of Winners 3:Joe Allen:NON","Choice_24_1:City of Portland, Councilor, District 1:3:Number of Winners 3:Joe Allen:NON","Choice_24_1:City of Portland, Councilor, District 1:4:Number of Winners 3:Joe Allen:NON","Choice_24_1:City of Portland, Councilor, District 1:5:Number of Winners 3:Joe Allen:NON","Choice_24_1:City of Portland, Councilor, District 1:6:Number of Winners 3:Joe Allen:NON","Choice_25_1:City of Portland, Councilor, District 1:1:Number of Winners 3:David Linn:NON","Choice_25_1:City of Portland, Councilor, District 1:2:Number of Winners 3:David Linn:NON","Choice_25_1:City of Portland, Councilor, District 1:3:Number of Winners 3:David Linn:NON","Choice_25_1:City of Portland, Councilor, District 1:4:Number of Winners 3:David Linn:NON","Choice_25_1:City of Portland, Councilor, District 1:5:Number of Winners 3:David Linn:NON","Choice_25_1:City of Portland, Councilor, District 1:6:Number of Winners 3:David Linn:NON","Choice_26_1:City of Portland, Councilor, District 1:1:Number of Winners 3:Cayle Tern:NON","Choice_26_1:City of Portland, Councilor, District 1:2:Number of Winners 3:Cayle Tern:NON","Choice_26_1:City of Portland, Councilor, District 1:3:Number of Winners 3:Cayle Tern:NON","Choice_26_1:City of Portland, Councilor, District 1:4:Number of Winners 3:Cayle Tern:NON","Choice_26_1:City of Portland, Councilor, District 1:5:Number of Winners 3:Cayle Tern:NON","Choice_26_1:City of Portland, Councilor, District 1:6:Number of Winners 3:Cayle Tern:NON","Choice_27_1

### Note that this is all the voters in the city, not just in D1.

In [4]:
list(D1_df.columns)

['RowNumber',
 'BoxID',
 'BoxPosition',
 'BallotID',
 'PrecinctID',
 'BallotStyleID',
 'PrecinctStyleName',
 'ScanComputerName',
 'Status',
 'Remade',
 'Choice_20_1:City of Portland, Councilor, District 1:1:Number of Winners 3:Peggy Sue Owens:NON',
 'Choice_20_1:City of Portland, Councilor, District 1:2:Number of Winners 3:Peggy Sue Owens:NON',
 'Choice_20_1:City of Portland, Councilor, District 1:3:Number of Winners 3:Peggy Sue Owens:NON',
 'Choice_20_1:City of Portland, Councilor, District 1:4:Number of Winners 3:Peggy Sue Owens:NON',
 'Choice_20_1:City of Portland, Councilor, District 1:5:Number of Winners 3:Peggy Sue Owens:NON',
 'Choice_20_1:City of Portland, Councilor, District 1:6:Number of Winners 3:Peggy Sue Owens:NON',
 'Choice_21_1:City of Portland, Councilor, District 1:1:Number of Winners 3:Timur Ender:NON',
 'Choice_21_1:City of Portland, Councilor, District 1:2:Number of Winners 3:Timur Ender:NON',
 'Choice_21_1:City of Portland, Councilor, District 1:3:Number of Winners

### Here are all the candidates that the CVR contains rankings for:

In [5]:
D1_candidates = [
    'Candace Avalos',
    'Jamie Dunphy',
    'Loretta Smith',
    'Noah Ernst',
    'Terrence Hayes',
    'Steph Routh',
    'Timur Ender',
    'Doug Clove',
    'Peggy Sue Owens',
    'David Linn',
    'Joe Allen',
    'Michael (Mike) Sands',
    'Deian Salazar',
    'Cayle Tern',
    'Thomas Shervey',
    'Joe Furi',
    'Uncertified Write In',
    'Write-in-120',
    'Write-in-121',
    'Write-in-122'
]

### Collect columns where 1st place, etc. votes are tallied so we can look for overvotes/undervotes while converting to a better format.

In [6]:
rank1_cols = [col for col in D1_df.columns if '1:Number' in col]
rank2_cols = [col for col in D1_df.columns if '2:Number' in col]
rank3_cols = [col for col in D1_df.columns if '3:Number' in col]
rank4_cols = [col for col in D1_df.columns if '4:Number' in col]
rank5_cols = [col for col in D1_df.columns if '5:Number' in col]
rank6_cols = [col for col in D1_df.columns if '6:Number' in col]

all_rank_cols = rank1_cols + rank2_cols + rank3_cols + rank4_cols + rank5_cols + rank6_cols

### This dataset has rows for ALL Portland voters; let's trim it down to voters who cast at least one vote.

In [7]:
D1_voters_df = D1_df[D1_df[all_rank_cols].sum(axis=1) > 0].reset_index(drop=True)

In [8]:
D1_voters_df 

,RowNumber,BoxID,BoxPosition,BallotID,PrecinctID,BallotStyleID,PrecinctStyleName,ScanComputerName,Status,Remade,"Choice_20_1:City of Portland, Councilor, District 1:1:Number of Winners 3:Peggy Sue Owens:NON","Choice_20_1:City of Portland, Councilor, District 1:2:Number of Winners 3:Peggy Sue Owens:NON","Choice_20_1:City of Portland, Councilor, District 1:3:Number of Winners 3:Peggy Sue Owens:NON","Choice_20_1:City of Portland, Councilor, District 1:4:Number of Winners 3:Peggy Sue Owens:NON","Choice_20_1:City of Portland, Councilor, District 1:5:Number of Winners 3:Peggy Sue Owens:NON","Choice_20_1:City of Portland, Councilor, District 1:6:Number of Winners 3:Peggy Sue Owens:NON","Choice_21_1:City of Portland, Councilor, District 1:1:Number of Winners 3:Timur Ender:NON","Choice_21_1:City of Portland, Councilor, District 1:2:Number of Winners 3:Timur Ender:NON","Choice_21_1:City of Portland, Councilor, District 1:3:Number of Winners 3:Timur Ender:NON","Choice_21_1:City of Portland, Councilor, District 1:4:Number of Winners 3:Timur Ender:NON","Choice_21_1:City of Portland, Councilor, District 1:5:Number of Winners 3:Timur Ender:NON","Choice_21_1:City of Portland, Councilor, District 1:6:Number of Winners 3:Timur Ender:NON","Choice_22_1:City of Portland, Councilor, District 1:1:Number of Winners 3:Noah Ernst:NON","Choice_22_1:City of Portland, Councilor, District 1:2:Number of Winners 3:Noah Ernst:NON","Choice_22_1:City of Portland, Councilor, District 1:3:Number of Winners 3:Noah Ernst:NON","Choice_22_1:City of Portland, Councilor, District 1:4:Number of Winners 3:Noah Ernst:NON","Choice_22_1:City of Portland, Councilor, District 1:5:Number of Winners 3:Noah Ernst:NON","Choice_22_1:City of Portland, Councilor, District 1:6:Number of Winners 3:Noah Ernst:NON","Choice_23_1:City of Portland, Councilor, District 1:1:Number of Winners 3:Candace Avalos:NON","Choice_23_1:City of Portland, Councilor, District 1:2:Number of Winners 3:Candace Avalos:NON","Choice_23_1:City of Portland, Councilor, District 1:3:Number of Winners 3:Candace Avalos:NON","Choice_23_1:City of Portland, Councilor, District 1:4:Number of Winners 3:Candace Avalos:NON","Choice_23_1:City of Portland, Councilor, District 1:5:Number of Winners 3:Candace Avalos:NON","Choice_23_1:City of Portland, Councilor, District 1:6:Number of Winners 3:Candace Avalos:NON","Choice_24_1:City of Portland, Councilor, District 1:1:Number of Winners 3:Joe Allen:NON","Choice_24_1:City of Portland, Councilor, District 1:2:Number of Winners 3:Joe Allen:NON","Choice_24_1:City of Portland, Councilor, District 1:3:Number of Winners 3:Joe Allen:NON","Choice_24_1:City of Portland, Councilor, District 1:4:Number of Winners 3:Joe Allen:NON","Choice_24_1:City of Portland, Councilor, District 1:5:Number of Winners 3:Joe Allen:NON","Choice_24_1:City of Portland, Councilor, District 1:6:Number of Winners 3:Joe Allen:NON","Choice_25_1:City of Portland, Councilor, District 1:1:Number of Winners 3:David Linn:NON","Choice_25_1:City of Portland, Councilor, District 1:2:Number of Winners 3:David Linn:NON","Choice_25_1:City of Portland, Councilor, District 1:3:Number of Winners 3:David Linn:NON","Choice_25_1:City of Portland, Councilor, District 1:4:Number of Winners 3:David Linn:NON","Choice_25_1:City of Portland, Councilor, District 1:5:Number of Winners 3:David Linn:NON","Choice_25_1:City of Portland, Councilor, District 1:6:Number of Winners 3:David Linn:NON","Choice_26_1:City of Portland, Councilor, District 1:1:Number of Winners 3:Cayle Tern:NON","Choice_26_1:City of Portland, Councilor, District 1:2:Number of Winners 3:Cayle Tern:NON","Choice_26_1:City of Portland, Councilor, District 1:3:Number of Winners 3:Cayle Tern:NON","Choice_26_1:City of Portland, Councilor, District 1:4:Number of Winners 3:Cayle Tern:NON","Choice_26_1:City of Portland, Councilor, District 1:5:Number of Winners 3:Cayle Tern:NON","Choice_26_1:City of Portland, Councilor, District 1:6:Number of Winners 3:Cayle Tern:NON","Choice_27_1

In [9]:
sorted(list(set(D1_voters_df['PrecinctStyleName'])))

['4406-2',
 '4407-1',
 '4508-1',
 '4509-1',
 '4605-2',
 '4606-2',
 '4607-1',
 '4702-1',
 '4703-1',
 '4704-1',
 '4705-1',
 '4706-1',
 '4707-1',
 '4708-1',
 '4709-1',
 '4710-1',
 '4801-1',
 '4802-1',
 '4803-1',
 '4804-1',
 '4805-1',
 '4806-1',
 '4809-1',
 '4910-1',
 '5001-1',
 '5003-1',
 'CLACK-113-1',
 'CLACK-114-1']

### So this file DOES include voters in Clackamas County.  (Washington County will only be relevant in D4.)

### Add a column for each raw ranking:

In [10]:
D1_voters_df.insert(10, "Rank1 raw", None)
D1_voters_df.insert(11, "Rank2 raw", None)
D1_voters_df.insert(12, "Rank3 raw", None)
D1_voters_df.insert(13, "Rank4 raw", None)
D1_voters_df.insert(14, "Rank5 raw", None)
D1_voters_df.insert(15, "Rank6 raw", None)


### For each voter and each rank, record whether the voter overvoted or undervoted in that rank, or if they voted for exactly one candidate in that rank, record that candidate.

In [11]:
for i in tqdm(D1_voters_df.index):
    
    rank1_votes = D1_voters_df.loc[i, rank1_cols].sum()
    if rank1_votes == 0:
        D1_voters_df.loc[i, 'Rank1 raw'] = "skipped"
    elif rank1_votes > 1:
        D1_voters_df.loc[i, 'Rank1 raw'] = "overvote"
    elif rank1_votes == 1:
        this_vote_col = [col for col in rank1_cols if D1_voters_df.loc[i, col] == 1][0]
        this_vote = [name for name in D1_candidates if name in this_vote_col][0]
        D1_voters_df.loc[i, 'Rank1 raw'] = this_vote

    rank2_votes = D1_voters_df.loc[i, rank2_cols].sum()
    if rank2_votes == 0:
        D1_voters_df.loc[i, 'Rank2 raw'] = "skipped"
    elif rank2_votes > 1:
        D1_voters_df.loc[i, 'Rank2 raw'] = "overvote"
    elif rank2_votes == 1:
        this_vote_col = [col for col in rank2_cols if D1_voters_df.loc[i, col] == 1][0]
        this_vote = [name for name in D1_candidates if name in this_vote_col][0]
        D1_voters_df.loc[i, 'Rank2 raw'] = this_vote

    rank3_votes = D1_voters_df.loc[i, rank3_cols].sum()
    if rank3_votes == 0:
        D1_voters_df.loc[i, 'Rank3 raw'] = "skipped"
    elif rank3_votes > 1:
        D1_voters_df.loc[i, 'Rank3 raw'] = "overvote"
    elif rank3_votes == 1:
        this_vote_col = [col for col in rank3_cols if D1_voters_df.loc[i, col] == 1][0]
        this_vote = [name for name in D1_candidates if name in this_vote_col][0]
        D1_voters_df.loc[i, 'Rank3 raw'] = this_vote

    rank4_votes = D1_voters_df.loc[i, rank4_cols].sum()
    if rank4_votes == 0:
        D1_voters_df.loc[i, 'Rank4 raw'] = "skipped"
    elif rank4_votes > 1:
        D1_voters_df.loc[i, 'Rank4 raw'] = "overvote"
    elif rank4_votes == 1:
        this_vote_col = [col for col in rank4_cols if D1_voters_df.loc[i, col] == 1][0]
        this_vote = [name for name in D1_candidates if name in this_vote_col][0]
        D1_voters_df.loc[i, 'Rank4 raw'] = this_vote

    rank5_votes = D1_voters_df.loc[i, rank5_cols].sum()
    if rank5_votes == 0:
        D1_voters_df.loc[i, 'Rank5 raw'] = "skipped"
    elif rank5_votes > 1:
        D1_voters_df.loc[i, 'Rank5 raw'] = "overvote"
    elif rank5_votes == 1:
        this_vote_col = [col for col in rank5_cols if D1_voters_df.loc[i, col] == 1][0]
        this_vote = [name for name in D1_candidates if name in this_vote_col][0]
        D1_voters_df.loc[i, 'Rank5 raw'] = this_vote

    rank6_votes = D1_voters_df.loc[i, rank6_cols].sum()
    if rank6_votes == 0:
        D1_voters_df.loc[i, 'Rank6 raw'] = "skipped"
    elif rank6_votes > 1:
        D1_voters_df.loc[i, 'Rank6 raw'] = "overvote"
    elif rank6_votes == 1:
        this_vote_col = [col for col in rank6_cols if D1_voters_df.loc[i, col] == 1][0]
        this_vote = [name for name in D1_candidates if name in this_vote_col][0]
        D1_voters_df.loc[i, 'Rank6 raw'] = this_vote


100%|████████████████████████████████████| 43669/43669 [01:16<00:00, 572.91it/s]


In [12]:
D1_voters_df

,RowNumber,BoxID,BoxPosition,BallotID,PrecinctID,BallotStyleID,PrecinctStyleName,ScanComputerName,Status,Remade,Rank1 raw,Rank2 raw,Rank3 raw,Rank4 raw,Rank5 raw,Rank6 raw,"Choice_20_1:City of Portland, Councilor, District 1:1:Number of Winners 3:Peggy Sue Owens:NON","Choice_20_1:City of Portland, Councilor, District 1:2:Number of Winners 3:Peggy Sue Owens:NON","Choice_20_1:City of Portland, Councilor, District 1:3:Number of Winners 3:Peggy Sue Owens:NON","Choice_20_1:City of Portland, Councilor, District 1:4:Number of Winners 3:Peggy Sue Owens:NON","Choice_20_1:City of Portland, Councilor, District 1:5:Number of Winners 3:Peggy Sue Owens:NON","Choice_20_1:City of Portland, Councilor, District 1:6:Number of Winners 3:Peggy Sue Owens:NON","Choice_21_1:City of Portland, Councilor, District 1:1:Number of Winners 3:Timur Ender:NON","Choice_21_1:City of Portland, Councilor, District 1:2:Number of Winners 3:Timur Ender:NON","Choice_21_1:City of Portland, Councilor, District 1:3:Number of Winners 3:Timur Ender:NON","Choice_21_1:City of Portland, Councilor, District 1:4:Number of Winners 3:Timur Ender:NON","Choice_21_1:City of Portland, Councilor, District 1:5:Number of Winners 3:Timur Ender:NON","Choice_21_1:City of Portland, Councilor, District 1:6:Number of Winners 3:Timur Ender:NON","Choice_22_1:City of Portland, Councilor, District 1:1:Number of Winners 3:Noah Ernst:NON","Choice_22_1:City of Portland, Councilor, District 1:2:Number of Winners 3:Noah Ernst:NON","Choice_22_1:City of Portland, Councilor, District 1:3:Number of Winners 3:Noah Ernst:NON","Choice_22_1:City of Portland, Councilor, District 1:4:Number of Winners 3:Noah Ernst:NON","Choice_22_1:City of Portland, Councilor, District 1:5:Number of Winners 3:Noah Ernst:NON","Choice_22_1:City of Portland, Councilor, District 1:6:Number of Winners 3:Noah Ernst:NON","Choice_23_1:City of Portland, Councilor, District 1:1:Number of Winners 3:Candace Avalos:NON","Choice_23_1:City of Portland, Councilor, District 1:2:Number of Winners 3:Candace Avalos:NON","Choice_23_1:City of Portland, Councilor, District 1:3:Number of Winners 3:Candace Avalos:NON","Choice_23_1:City of Portland, Councilor, District 1:4:Number of Winners 3:Candace Avalos:NON","Choice_23_1:City of Portland, Councilor, District 1:5:Number of Winners 3:Candace Avalos:NON","Choice_23_1:City of Portland, Councilor, District 1:6:Number of Winners 3:Candace Avalos:NON","Choice_24_1:City of Portland, Councilor, District 1:1:Number of Winners 3:Joe Allen:NON","Choice_24_1:City of Portland, Councilor, District 1:2:Number of Winners 3:Joe Allen:NON","Choice_24_1:City of Portland, Councilor, District 1:3:Number of Winners 3:Joe Allen:NON","Choice_24_1:City of Portland, Councilor, District 1:4:Number of Winners 3:Joe Allen:NON","Choice_24_1:City of Portland, Councilor, District 1:5:Number of Winners 3:Joe Allen:NON","Choice_24_1:City of Portland, Councilor, District 1:6:Number of Winners 3:Joe Allen:NON","Choice_25_1:City of Portland, Councilor, District 1:1:Number of Winners 3:David Linn:NON","Choice_25_1:City of Portland, Councilor, District 1:2:Number of Winners 3:David Linn:NON","Choice_25_1:City of Portland, Councilor, District 1:3:Number of Winners 3:David Linn:NON","Choice_25_1:City of Portland, Councilor, District 1:4:Number of Winners 3:David Linn:NON","Choice_25_1:City of Portland, Councilor, District 1:5:Number of Winners 3:David Linn:NON","Choice_25_1:City of Portland, Councilor, District 1:6:Number of Winners 3:David Linn:NON","Choice_26_1:City of Portland, Councilor, District 1:1:Number of Winners 3:Cayle Tern:NON","Choice_26_1:City of Portland, Councilor, District 1:2:Number of Winners 3:Cayle Tern:NON","Choice_26_1:City of Portland, Councilor, District 1:3:Number of Winners 3:Cayle Tern:NON","Choice_26_1:City of Portland, Councilor, District 1:4:Number of Winners 3:Cayle Tern:NON","Choice_26_1:City of Portland, Councilor, District 1:5:Number of Winners 3:Cayle Tern:NON","Choice_26_1:City of Portland, Councilor, D

### Next, clean the votes for VoteKit according to Portland's rules: overvotes and undervotes are both skipped and ballots are transferred to the next ranking.  If the same candidate receives multiple rankings, only keep the highest ranking.
### It appears that the categories 'Write-in-120', 'Write-in-121', and 'Write-in-122' are ALSO skipped and ballots are transferred to the next ranking.  (Equivalently, these "candidates" were eliminated first in a "Round 0" that occurred before the published tabulation rounds.)

In [13]:
D1_voters_df.insert(16, "Rank1 condensed", None)
D1_voters_df.insert(17, "Rank2 condensed", None)
D1_voters_df.insert(18, "Rank3 condensed", None)
D1_voters_df.insert(19, "Rank4 condensed", None)
D1_voters_df.insert(20, "Rank5 condensed", None)
D1_voters_df.insert(21, "Rank6 condensed", None)


### While we do this, save the condensed ballots to a list to run the election with.

In [14]:
D1_candidates_R1 = D1_candidates[:-3]
D1_candidates_R1

['Candace Avalos',
 'Jamie Dunphy',
 'Loretta Smith',
 'Noah Ernst',
 'Terrence Hayes',
 'Steph Routh',
 'Timur Ender',
 'Doug Clove',
 'Peggy Sue Owens',
 'David Linn',
 'Joe Allen',
 'Michael (Mike) Sands',
 'Deian Salazar',
 'Cayle Tern',
 'Thomas Shervey',
 'Joe Furi',
 'Uncertified Write In']

In [15]:
condensed_ballots_R1 = []

for i in tqdm(D1_voters_df.index):
    this_ballot_raw = D1_voters_df.loc[i, ['Rank1 raw', 'Rank2 raw', 'Rank3 raw', 'Rank4 raw', 'Rank5 raw', 'Rank6 raw']]
    
    this_ballot_cleaned = [name for name in this_ballot_raw if name in D1_candidates_R1]

    this_ballot_condensed = []
    for name in this_ballot_cleaned:
        if name not in this_ballot_condensed:
            this_ballot_condensed.append(name)    

    if len(this_ballot_condensed) > 0:
        condensed_ballots_R1.append(this_ballot_condensed)
    
    if len(this_ballot_condensed) < 6:
        fill_len = 6 - len(this_ballot_condensed)
        this_ballot_condensed = this_ballot_condensed + ["skipped"]*fill_len

    D1_voters_df.loc[i, "Rank1 condensed"] = this_ballot_condensed[0]
    D1_voters_df.loc[i, "Rank2 condensed"] = this_ballot_condensed[1]
    D1_voters_df.loc[i, "Rank3 condensed"] = this_ballot_condensed[2]
    D1_voters_df.loc[i, "Rank4 condensed"] = this_ballot_condensed[3]
    D1_voters_df.loc[i, "Rank5 condensed"] = this_ballot_condensed[4]
    D1_voters_df.loc[i, "Rank6 condensed"] = this_ballot_condensed[5]
    

100%|███████████████████████████████████| 43669/43669 [00:18<00:00, 2322.63it/s]


### How many first-round votes did each candidate get?

In [16]:
round_1_votes = {}
for name in D1_candidates_R1:
    round_1_votes[name] = 0

for ballot in condensed_ballots_R1:
    round_1_votes[ballot[0]] += 1

In [17]:
for name in D1_candidates_R1:
    print(name, round_1_votes[name])


Candace Avalos 8297
Jamie Dunphy 5064
Loretta Smith 5586
Noah Ernst 4052
Terrence Hayes 3975
Steph Routh 3894
Timur Ender 3550
Doug Clove 1698
Peggy Sue Owens 1266
David Linn 1111
Joe Allen 978
Michael (Mike) Sands 952
Deian Salazar 720
Cayle Tern 711
Thomas Shervey 385
Joe Furi 355
Uncertified Write In 277


### Excellent - we got the algorithm right!

### How many voters voted in the first round?

In [18]:
len(condensed_ballots_R1)

42871

### Just for kicks, check the round 2 tallies too.

In [19]:
D1_candidates_R2 = D1_candidates_R1[:-1]

In [20]:
D1_candidates_R2

['Candace Avalos',
 'Jamie Dunphy',
 'Loretta Smith',
 'Noah Ernst',
 'Terrence Hayes',
 'Steph Routh',
 'Timur Ender',
 'Doug Clove',
 'Peggy Sue Owens',
 'David Linn',
 'Joe Allen',
 'Michael (Mike) Sands',
 'Deian Salazar',
 'Cayle Tern',
 'Thomas Shervey',
 'Joe Furi']

In [21]:
condensed_ballots_R2 = []

for ballot in tqdm(condensed_ballots_R1):
    new_ballot = [name for name in ballot if name in D1_candidates_R2]
    if len(new_ballot) > 0:
        condensed_ballots_R2.append(new_ballot)

100%|█████████████████████████████████| 42871/42871 [00:00<00:00, 630718.31it/s]


### Round 2 votes:

In [22]:
round_2_votes = {}
for name in D1_candidates_R2:
    round_2_votes[name] = 0

for ballot in condensed_ballots_R2:
    round_2_votes[ballot[0]] += 1

In [23]:
for name in D1_candidates_R2:
    print(name, round_2_votes[name])


Candace Avalos 8307
Jamie Dunphy 5068
Loretta Smith 5595
Noah Ernst 4060
Terrence Hayes 3984
Steph Routh 3898
Timur Ender 3556
Doug Clove 1709
Peggy Sue Owens 1269
David Linn 1113
Joe Allen 981
Michael (Mike) Sands 957
Deian Salazar 720
Cayle Tern 713
Thomas Shervey 387
Joe Furi 361


### These are correct too!

### So, let's create two datasets to save: one with the raw data and one with the condensed data.  Keep all the intro columns for the raw data, but not for the cleaned data.

In [24]:
D1_voters_df

,RowNumber,BoxID,BoxPosition,BallotID,PrecinctID,BallotStyleID,PrecinctStyleName,ScanComputerName,Status,Remade,Rank1 raw,Rank2 raw,Rank3 raw,Rank4 raw,Rank5 raw,Rank6 raw,Rank1 condensed,Rank2 condensed,Rank3 condensed,Rank4 condensed,Rank5 condensed,Rank6 condensed,"Choice_20_1:City of Portland, Councilor, District 1:1:Number of Winners 3:Peggy Sue Owens:NON","Choice_20_1:City of Portland, Councilor, District 1:2:Number of Winners 3:Peggy Sue Owens:NON","Choice_20_1:City of Portland, Councilor, District 1:3:Number of Winners 3:Peggy Sue Owens:NON","Choice_20_1:City of Portland, Councilor, District 1:4:Number of Winners 3:Peggy Sue Owens:NON","Choice_20_1:City of Portland, Councilor, District 1:5:Number of Winners 3:Peggy Sue Owens:NON","Choice_20_1:City of Portland, Councilor, District 1:6:Number of Winners 3:Peggy Sue Owens:NON","Choice_21_1:City of Portland, Councilor, District 1:1:Number of Winners 3:Timur Ender:NON","Choice_21_1:City of Portland, Councilor, District 1:2:Number of Winners 3:Timur Ender:NON","Choice_21_1:City of Portland, Councilor, District 1:3:Number of Winners 3:Timur Ender:NON","Choice_21_1:City of Portland, Councilor, District 1:4:Number of Winners 3:Timur Ender:NON","Choice_21_1:City of Portland, Councilor, District 1:5:Number of Winners 3:Timur Ender:NON","Choice_21_1:City of Portland, Councilor, District 1:6:Number of Winners 3:Timur Ender:NON","Choice_22_1:City of Portland, Councilor, District 1:1:Number of Winners 3:Noah Ernst:NON","Choice_22_1:City of Portland, Councilor, District 1:2:Number of Winners 3:Noah Ernst:NON","Choice_22_1:City of Portland, Councilor, District 1:3:Number of Winners 3:Noah Ernst:NON","Choice_22_1:City of Portland, Councilor, District 1:4:Number of Winners 3:Noah Ernst:NON","Choice_22_1:City of Portland, Councilor, District 1:5:Number of Winners 3:Noah Ernst:NON","Choice_22_1:City of Portland, Councilor, District 1:6:Number of Winners 3:Noah Ernst:NON","Choice_23_1:City of Portland, Councilor, District 1:1:Number of Winners 3:Candace Avalos:NON","Choice_23_1:City of Portland, Councilor, District 1:2:Number of Winners 3:Candace Avalos:NON","Choice_23_1:City of Portland, Councilor, District 1:3:Number of Winners 3:Candace Avalos:NON","Choice_23_1:City of Portland, Councilor, District 1:4:Number of Winners 3:Candace Avalos:NON","Choice_23_1:City of Portland, Councilor, District 1:5:Number of Winners 3:Candace Avalos:NON","Choice_23_1:City of Portland, Councilor, District 1:6:Number of Winners 3:Candace Avalos:NON","Choice_24_1:City of Portland, Councilor, District 1:1:Number of Winners 3:Joe Allen:NON","Choice_24_1:City of Portland, Councilor, District 1:2:Number of Winners 3:Joe Allen:NON","Choice_24_1:City of Portland, Councilor, District 1:3:Number of Winners 3:Joe Allen:NON","Choice_24_1:City of Portland, Councilor, District 1:4:Number of Winners 3:Joe Allen:NON","Choice_24_1:City of Portland, Councilor, District 1:5:Number of Winners 3:Joe Allen:NON","Choice_24_1:City of Portland, Councilor, District 1:6:Number of Winners 3:Joe Allen:NON","Choice_25_1:City of Portland, Councilor, District 1:1:Number of Winners 3:David Linn:NON","Choice_25_1:City of Portland, Councilor, District 1:2:Number of Winners 3:David Linn:NON","Choice_25_1:City of Portland, Councilor, District 1:3:Number of Winners 3:David Linn:NON","Choice_25_1:City of Portland, Councilor, District 1:4:Number of Winners 3:David Linn:NON","Choice_25_1:City of Portland, Councilor, District 1:5:Number of Winners 3:David Linn:NON","Choice_25_1:City of Portland, Councilor, District 1:6:Number of Winners 3:David Linn:NON","Choice_26_1:City of Portland, Councilor, District 1:1:Number of Winners 3:Cayle Tern:NON","Choice_26_1:City of Portland, Councilor, District 1:2:Number of Winners 3:Cayle Tern:NON","Choice_26_1:City of Portland, Councilor, District 1:3:Number of Winners 3:Cayle Tern:NON","Choice_26_1:City of Portland, Councilor, District 1:4:Number of Winners 3:Cayle Tern:NON","Choice_26_1:City of Portland, Council

In [25]:
D1_voters_df.columns[0:16]

Index(['RowNumber', 'BoxID', 'BoxPosition', 'BallotID', 'PrecinctID',
       'BallotStyleID', 'PrecinctStyleName', 'ScanComputerName', 'Status',
       'Remade', 'Rank1 raw', 'Rank2 raw', 'Rank3 raw', 'Rank4 raw',
       'Rank5 raw', 'Rank6 raw'],
      dtype='object')

In [26]:
D1_raw_df = D1_voters_df[D1_voters_df.columns[0:16]]

In [27]:
D1_voters_df.columns[16:22]

Index(['Rank1 condensed', 'Rank2 condensed', 'Rank3 condensed',
       'Rank4 condensed', 'Rank5 condensed', 'Rank6 condensed'],
      dtype='object')

In [28]:
D1_condensed_df = D1_voters_df[['PrecinctStyleName'] + list(D1_voters_df.columns)[16:22]]

### The raw file is good to go; I just want to rename the ranking columns.

In [29]:
D1_raw_df = D1_raw_df.rename(columns = {
    'Rank1 raw' : 'rank1',
    'Rank2 raw' : 'rank2',
    'Rank3 raw' : 'rank3',
    'Rank4 raw' : 'rank4',
    'Rank5 raw' : 'rank5',
    'Rank6 raw' : 'rank6'
})

In [30]:
D1_raw_df

,RowNumber,BoxID,BoxPosition,BallotID,PrecinctID,BallotStyleID,PrecinctStyleName,ScanComputerName,Status,Remade,rank1,rank2,rank3,rank4,rank5,rank6
0,54,RCV-0001,54,RCV-0001+10109,35,2,4703-1,ScanStation6,0,0,Terrence Hayes,Loretta Smith,Noah Ernst,skipped,skipped,skipped
1,58,RCV-0001,58,RCV-0001+10117,20,2,4508-1,ScanStation6,0,0,Loretta Smith,Steph Routh,Timur Ender,David Linn,Candace Avalos,Jamie Dunphy
2,59,RCV-0001,59,RCV-0001+10119,20,2,4508-1,ScanStation6,0,0,Loretta Smith,Steph Routh,Timur Ender,David Linn,Candace Avalos,Jamie Dunphy
3,124,RCV-0001,124,RCV-0001+10249,9,2,4802-1,ScanStation6,0,0,Michael (Mike) Sands,Doug Clove,Joe Furi,Timur Ender,Deian Salazar,Loretta Smith
4,199,RCV-0002,11,RCV-0002+10023,39,2,4707-1,ScanStation8,0,0,Timur Ender,Candace Avalos,Cayle Tern,Steph Routh,Michael (Mike) Sands,Jamie Dunphy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43664,332933,RCV-1943,11,RCV-1943+10023,41,2,4509-1,ScanStation5,0,0,Loretta Smith,Candace Avalos,Jamie Dunphy,Timur Ender,skipped,skipped
43665,332938,RCV-1943,16,RCV-1943+10033,18,2,4705-1,ScanStation5,0,0,Loretta Smith,skipped,skipped,skipped,skipped,skipped
43666,332950,RCV-1944,11,RCV-1944+10023,13,2,4708-1,ScanStation6,-1,1,Steph Routh,Jamie Dunphy,skipped,skipped,skipped,skipped
43667,332963,RCV-1944,24,RCV-1944+10049,24,2,4806-1,ScanStation6,0,0,Steph Routh,Loretta Smith,Candace Avalos,skipped,skipped,skipped


In [31]:
D1_raw_df.to_csv("./Data/Cast Vote Records/Cleaned CVRs/Portland_D1_Raw.csv", index = False)

### For the condensed file, I'd like to remove voters who no longer have any votes. (These are the ones who only voted for 'Write-in-120', 'Write-in-121', and/or 'Write-in-122'.)  
### Check that this produces the correct number of first-round votes:

In [32]:
len(D1_condensed_df[D1_condensed_df['Rank1 condensed'] != 'skipped'])

42871

In [33]:
D1_condensed_df = D1_condensed_df[D1_condensed_df['Rank1 condensed'] != 'skipped'].reset_index(drop="True")

### Rename the columns:

In [34]:
D1_condensed_df = D1_condensed_df.rename(columns = {
    'PrecinctStyleName': 'Precinct',
    'Rank1 condensed' : 'rank1',
    'Rank2 condensed' : 'rank2',
    'Rank3 condensed' : 'rank3',
    'Rank4 condensed' : 'rank4',
    'Rank5 condensed' : 'rank5',
    'Rank6 condensed' : 'rank6'
})

In [35]:
D1_condensed_df

,Precinct,rank1,rank2,rank3,rank4,rank5,rank6
0,4703-1,Terrence Hayes,Loretta Smith,Noah Ernst,skipped,skipped,skipped
1,4508-1,Loretta Smith,Steph Routh,Timur Ender,David Linn,Candace Avalos,Jamie Dunphy
2,4508-1,Loretta Smith,Steph Routh,Timur Ender,David Linn,Candace Avalos,Jamie Dunphy
3,4802-1,Michael (Mike) Sands,Doug Clove,Joe Furi,Timur Ender,Deian Salazar,Loretta Smith
4,4707-1,Timur Ender,Candace Avalos,Cayle Tern,Steph Routh,Michael (Mike) Sands,Jamie Dunphy
...,...,...,...,...,...,...,...
42866,4509-1,Loretta Smith,Candace Avalos,Jamie Dunphy,Timur Ender,skipped,skipped
42867,4705-1,Loretta Smith,skipped,skipped,skipped,skipped,skipped
42868,4708-1,Steph Routh,Jamie Dunphy,skipped,skipped,skipped,skipped
42869,4806-1,Steph Routh,Loretta Smith,Candace Avalos,skipped,skipped,skipped


### Change the format of the entries in the Precinct column to match the precinct shapefile:

In [36]:
set(D1_condensed_df['Precinct'])

{'4406-2',
 '4407-1',
 '4508-1',
 '4509-1',
 '4605-2',
 '4606-2',
 '4607-1',
 '4702-1',
 '4703-1',
 '4704-1',
 '4705-1',
 '4706-1',
 '4707-1',
 '4708-1',
 '4709-1',
 '4710-1',
 '4801-1',
 '4802-1',
 '4803-1',
 '4804-1',
 '4805-1',
 '4806-1',
 '4809-1',
 '4910-1',
 '5001-1',
 '5003-1',
 'CLACK-113-1',
 'CLACK-114-1'}

In [37]:
for i in D1_condensed_df.index:
    if D1_condensed_df.loc[i, 'Precinct'].startswith('CLACK'):
        D1_condensed_df.loc[i, 'Precinct'] = 'C'+ D1_condensed_df.loc[i, 'Precinct'][-5:-2]
    elif D1_condensed_df.loc[i, 'Precinct'].startswith('WASH'):
        D1_condensed_df.loc[i, 'Precinct'] = 'W'+ D1_condensed_df.loc[i, 'Precinct'][-5:-2]
    else:
        D1_condensed_df.loc[i, 'Precinct'] = 'M'+ D1_condensed_df.loc[i, 'Precinct'][0:4]

In [38]:
set(D1_condensed_df['Precinct'])

{'C113',
 'C114',
 'M4406',
 'M4407',
 'M4508',
 'M4509',
 'M4605',
 'M4606',
 'M4607',
 'M4702',
 'M4703',
 'M4704',
 'M4705',
 'M4706',
 'M4707',
 'M4708',
 'M4709',
 'M4710',
 'M4801',
 'M4802',
 'M4803',
 'M4804',
 'M4805',
 'M4806',
 'M4809',
 'M4910',
 'M5001',
 'M5003'}

In [39]:
D1_condensed_df

,Precinct,rank1,rank2,rank3,rank4,rank5,rank6
0,M4703,Terrence Hayes,Loretta Smith,Noah Ernst,skipped,skipped,skipped
1,M4508,Loretta Smith,Steph Routh,Timur Ender,David Linn,Candace Avalos,Jamie Dunphy
2,M4508,Loretta Smith,Steph Routh,Timur Ender,David Linn,Candace Avalos,Jamie Dunphy
3,M4802,Michael (Mike) Sands,Doug Clove,Joe Furi,Timur Ender,Deian Salazar,Loretta Smith
4,M4707,Timur Ender,Candace Avalos,Cayle Tern,Steph Routh,Michael (Mike) Sands,Jamie Dunphy
...,...,...,...,...,...,...,...
42866,M4509,Loretta Smith,Candace Avalos,Jamie Dunphy,Timur Ender,skipped,skipped
42867,M4705,Loretta Smith,skipped,skipped,skipped,skipped,skipped
42868,M4708,Steph Routh,Jamie Dunphy,skipped,skipped,skipped,skipped
42869,M4806,Steph Routh,Loretta Smith,Candace Avalos,skipped,skipped,skipped


### Save it!

In [40]:
D1_condensed_df.to_csv("./Data/Cast Vote Records/Cleaned CVRs/Portland_D1_Condensed.csv", index = False)

### Count ballot lengths for condensed ballots:

In [41]:
condensed_ballot_lengths = {1:0, 2:0, 3:0, 4:0, 5:0, 6:0}

for i in tqdm(D1_condensed_df.index):
    num_skipped = list(D1_condensed_df.loc[i]).count('skipped')
    num_ranked = 6 - num_skipped
    condensed_ballot_lengths[num_ranked] += 1
    

100%|█████████████████████████████████| 42871/42871 [00:00<00:00, 139764.27it/s]


In [42]:
condensed_ballot_lengths

{1: 5857, 2: 3072, 3: 8350, 4: 4831, 5: 3869, 6: 16892}

In [43]:
sum(condensed_ballot_lengths.values())

42871

In [44]:
condensed_ballot_length_pcts = {i: condensed_ballot_lengths[i]/sum(condensed_ballot_lengths.values()) for i in condensed_ballot_lengths.keys()}
condensed_ballot_length_pcts

{1: 0.1366191598049964,
 2: 0.07165683095799025,
 3: 0.19477035758438105,
 4: 0.11268689790301135,
 5: 0.09024748664598446,
 6: 0.3940192671036365}

### ONE MORE THING!  I want to create "viability-condensed" ballots that only keep the candidates who had enough MENTIONS to meet the quota for election.

### First we need to count mentions; we can use the condensed ballots for this.

In [45]:
mention_dict = {name: 0 for name in D1_candidates_R1}

for i in tqdm(D1_condensed_df.index):
    for rank in ['rank1', 'rank2', 'rank3', 'rank4', 'rank5', 'rank6']:
        if D1_condensed_df.loc[i, rank] != 'skipped':
            mention_dict[D1_condensed_df.loc[i, rank]] += 1

100%|██████████████████████████████████| 42871/42871 [00:00<00:00, 43463.32it/s]


In [46]:
mention_dict

{'Candace Avalos': 22267,
 'Jamie Dunphy': 18470,
 'Loretta Smith': 17984,
 'Noah Ernst': 11606,
 'Terrence Hayes': 15967,
 'Steph Routh': 20440,
 'Timur Ender': 16858,
 'Doug Clove': 7593,
 'Peggy Sue Owens': 3869,
 'David Linn': 9324,
 'Joe Allen': 4955,
 'Michael (Mike) Sands': 5196,
 'Deian Salazar': 5262,
 'Cayle Tern': 9061,
 'Thomas Shervey': 3507,
 'Joe Furi': 4070,
 'Uncertified Write In': 643}

In [47]:
quota = 10718  # D1 quota

In [48]:
viable_candidates = [name for name in mention_dict.keys() if mention_dict[name] >= quota]

In [49]:
viable_candidates

['Candace Avalos',
 'Jamie Dunphy',
 'Loretta Smith',
 'Noah Ernst',
 'Terrence Hayes',
 'Steph Routh',
 'Timur Ender']

### Now let's further condense ballots to only include the viable candidates.

In [50]:
D1_voters_df.insert(len(D1_voters_df.columns), "Rank1 viable-condensed", None)
D1_voters_df.insert(len(D1_voters_df.columns), "Rank2 viable-condensed", None)
D1_voters_df.insert(len(D1_voters_df.columns), "Rank3 viable-condensed", None)
D1_voters_df.insert(len(D1_voters_df.columns), "Rank4 viable-condensed", None)
D1_voters_df.insert(len(D1_voters_df.columns), "Rank5 viable-condensed", None)
D1_voters_df.insert(len(D1_voters_df.columns), "Rank6 viable-condensed", None)


In [51]:
for i in tqdm(D1_voters_df.index):
    this_ballot_condensed = D1_voters_df.loc[i, ['Rank1 condensed', 'Rank2 condensed', 'Rank3 condensed', 'Rank4 condensed', 'Rank5 condensed', 'Rank6 condensed']]

    this_ballot_viable_condensed = [name for name in this_ballot_condensed if name in viable_candidates]

    if len(this_ballot_viable_condensed) < 6:
        fill_len = 6 - len(this_ballot_viable_condensed)
        this_ballot_viable_condensed = this_ballot_viable_condensed + ["skipped"]*fill_len

    D1_voters_df.loc[i, "Rank1 viable-condensed"] = this_ballot_viable_condensed[0]
    D1_voters_df.loc[i, "Rank2 viable-condensed"] = this_ballot_viable_condensed[1]
    D1_voters_df.loc[i, "Rank3 viable-condensed"] = this_ballot_viable_condensed[2]
    D1_voters_df.loc[i, "Rank4 viable-condensed"] = this_ballot_viable_condensed[3]
    D1_voters_df.loc[i, "Rank5 viable-condensed"] = this_ballot_viable_condensed[4]
    D1_voters_df.loc[i, "Rank6 viable-condensed"] = this_ballot_viable_condensed[5]

100%|███████████████████████████████████| 43669/43669 [00:19<00:00, 2255.29it/s]


In [52]:
len(D1_voters_df.columns)

148

In [53]:
D1_voters_df.columns[142:148]

Index(['Rank1 viable-condensed', 'Rank2 viable-condensed',
       'Rank3 viable-condensed', 'Rank4 viable-condensed',
       'Rank5 viable-condensed', 'Rank6 viable-condensed'],
      dtype='object')

In [54]:
D1_viable_condensed_df = D1_voters_df[['PrecinctStyleName'] + list(D1_voters_df.columns)[142:148]]

### Rename the columns:

In [55]:
D1_viable_condensed_df = D1_viable_condensed_df.rename(columns = {
    'PrecinctStyleName': 'Precinct',
    'Rank1 viable-condensed' : 'rank1',
    'Rank2 viable-condensed' : 'rank2',
    'Rank3 viable-condensed' : 'rank3',
    'Rank4 viable-condensed' : 'rank4',
    'Rank5 viable-condensed' : 'rank5',
    'Rank6 viable-condensed' : 'rank6'
})

### Change the format of the entries in the Precinct column to match the precinct shapefile:

In [56]:
set(D1_viable_condensed_df['Precinct'])

{'4406-2',
 '4407-1',
 '4508-1',
 '4509-1',
 '4605-2',
 '4606-2',
 '4607-1',
 '4702-1',
 '4703-1',
 '4704-1',
 '4705-1',
 '4706-1',
 '4707-1',
 '4708-1',
 '4709-1',
 '4710-1',
 '4801-1',
 '4802-1',
 '4803-1',
 '4804-1',
 '4805-1',
 '4806-1',
 '4809-1',
 '4910-1',
 '5001-1',
 '5003-1',
 'CLACK-113-1',
 'CLACK-114-1'}

In [57]:
for i in D1_viable_condensed_df.index:
    if D1_viable_condensed_df.loc[i, 'Precinct'].startswith('CLACK'):
        D1_viable_condensed_df.loc[i, 'Precinct'] = 'C'+ D1_viable_condensed_df.loc[i, 'Precinct'][-5:-2]
    elif D1_viable_condensed_df.loc[i, 'Precinct'].startswith('WASH'):
        D1_viable_condensed_df.loc[i, 'Precinct'] = 'W'+ D1_viable_condensed_df.loc[i, 'Precinct'][-5:-2]
    else:
        D1_viable_condensed_df.loc[i, 'Precinct'] = 'M'+ D1_viable_condensed_df.loc[i, 'Precinct'][0:4]

In [58]:
set(D1_viable_condensed_df['Precinct'])

{'C113',
 'C114',
 'M4406',
 'M4407',
 'M4508',
 'M4509',
 'M4605',
 'M4606',
 'M4607',
 'M4702',
 'M4703',
 'M4704',
 'M4705',
 'M4706',
 'M4707',
 'M4708',
 'M4709',
 'M4710',
 'M4801',
 'M4802',
 'M4803',
 'M4804',
 'M4805',
 'M4806',
 'M4809',
 'M4910',
 'M5001',
 'M5003'}

### Remove voters who no longer have votes:

In [59]:
D1_viable_condensed_df = D1_viable_condensed_df[D1_viable_condensed_df['rank1'] != 'skipped'].reset_index(drop="True")

In [60]:
D1_viable_condensed_df

,Precinct,rank1,rank2,rank3,rank4,rank5,rank6
0,M4703,Terrence Hayes,Loretta Smith,Noah Ernst,skipped,skipped,skipped
1,M4508,Loretta Smith,Steph Routh,Timur Ender,Candace Avalos,Jamie Dunphy,skipped
2,M4508,Loretta Smith,Steph Routh,Timur Ender,Candace Avalos,Jamie Dunphy,skipped
3,M4802,Timur Ender,Loretta Smith,skipped,skipped,skipped,skipped
4,M4707,Timur Ender,Candace Avalos,Steph Routh,Jamie Dunphy,skipped,skipped
...,...,...,...,...,...,...,...
40489,M4509,Loretta Smith,Candace Avalos,Jamie Dunphy,Timur Ender,skipped,skipped
40490,M4705,Loretta Smith,skipped,skipped,skipped,skipped,skipped
40491,M4708,Steph Routh,Jamie Dunphy,skipped,skipped,skipped,skipped
40492,M4806,Steph Routh,Loretta Smith,Candace Avalos,skipped,skipped,skipped


### How many/what percentage of voters ranked at least one viable candidate?

In [61]:
print(len(D1_viable_condensed_df), len(D1_condensed_df), len(D1_viable_condensed_df)/len(D1_condensed_df))

40494 42871 0.944554594014602


### Save it!

In [62]:
D1_viable_condensed_df.to_csv("./Data/Cast Vote Records/Cleaned CVRs/Portland_D1_Viable_Condensed.csv", index = False)